<a href="https://colab.research.google.com/github/gabriellaaileen/ADL/blob/main/UTS%20ADL/UTS_ADL_Running_Modul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install future

In [2]:
!pip install rasterio
import rasterio

In [3]:
# A
import argparse
# C
import cv2
# G
import glob
# H
import h5py
# I
import imgaug as ia
import importlib.util
# J
import json
# M
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# N
import numpy as np
# O
import os
# P
import pandas as pd
# R
import random
import rasterio
# S
import scipy.misc
import shutil
import signal
import sys
# T
import tensorflow
import time
# W
import webbrowser

### Keras
from keras.layers import  * 
from keras.models import Model, Sequential, load_model as keras_load_model
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, CSVLogger, TensorBoard, EarlyStopping
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras import losses
from keras.optimizers import *
from keras.utils.generic_utils import get_custom_objects
### Tensor
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, RMSprop
### Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
### Random
from random import randint
from random import shuffle
### ImgAug
from imgaug import augmenters as iaa
from imgaug import parameters as iap
### Others
#from _future_ import division
from IPython.display import clear_output

In [4]:
!git clone https://github.com/gabriellaaileen/ADL.git
%cd UTS ADL

fatal: destination path 'ADL' already exists and is not an empty directory.
[Errno 2] No such file or directory: 'UTS ADL'
/content


In [5]:
%cd /content/ADL/UTS ADL

/content/ADL/UTS ADL


In [51]:
!python3 "/content/ADL/UTS ADL/generate_parameters.py"

Warning params file exists, continue? (y/n) y


In [52]:
!python3 "/content/ADL/UTS ADL/rgb2label.py"

In [53]:
!python3 "/content/ADL/UTS ADL/TrainingClass.py"

In [54]:
!python3 "/content/ADL/UTS ADL/PostProcessing.py"

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [55]:
!python3 "/content/ADL/UTS ADL/XNet.py"

In [56]:
!python3 "/content/ADL/UTS ADL/train.py"

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
I will train on all these parameter files:

parameters.txt
Warning, folder exists! Delete? (y/n) y
Traceback (most recent call last):
  File "/content/ADL/UTS ADL/train.py", line 41, in <module>
    training = TrainingClass.TrainingClass(**params)
  File "/content/ADL/UTS ADL/TrainingClass.py", line 59, in __init__
    self.load_data()
  File "/content/ADL/UTS ADL/TrainingClass.py", line 68, in load_data
    hf = h5py.File(self.data_path, 'r')
  File "/usr/local/lib/python3.7/dist-packages/h5py/_hl/files.py", line 427, in __init__
    swmr=swmr)
  File "/usr/local/lib/python3.7/dist-packages/h5py/_hl/files.py", line 190, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects

In [37]:
!python3 "/content/ADL/UTS ADL/augmentations.py"

Checking labels and data match in Humans folder ...
Traceback (most recent call last):
  File "/content/ADL/UTS ADL/augmentations.py", line 51, in <module>
    balanced_test_val_split(main_path, data_to_add, image_size, train_size, n_classes)
  File "/content/ADL/UTS ADL/utils.py", line 27, in balanced_test_val_split
    assert len(labels) != 0
AssertionError


In [42]:
import numpy as np
import TrainingClass
import json
import os
import sys
from PostProcessing import PostProcessing
import glob
import shutil
import webbrowser
import time
#from Killer import GracefulKiller
#killer = GracefulKiller()

param_files = glob.glob("aug*.txt")
#param_files = glob.glob("parameters.txt")
print("I will train on all these parameter files:\n")
print(*param_files, sep = "\n")

I will train on all these parameter files:




In [45]:
#print("Opening tensorboard... \n")
#tb_url = "http://127.0.0.1:7007/"
#webbrowser.open(tb_url)

for file in param_files:
    params = json.load(open(file,'r'))

    save_folder = params["save_folder"]
    if(os.path.isdir(save_folder)):
        rm_folder = input("Warning, folder exists! Delete? (y/n) ")
        if(rm_folder == "y"):
            shutil.rmtree(save_folder)
        else:
            sys.exit()
            
    os.mkdir(save_folder)
    training = TrainingClass.TrainingClass(**params)

In [46]:
#tensorboard stuff
#tbdir = os.path.join(save_folder, "tboard")
#os.mkdir(tbdir)
#os.system("killall tensorboard")
#os.system("tensorboard --logdir=" + tbdir + " --port=7007 &")

try:
  training.fit()
except:
  print("\n Dying... \n")


 Dying... 



In [41]:
   
    print("Running post training analysis...\n")
        
    h5_files = np.sort(glob.glob(os.path.join(params["save_folder"], "*.h5")))
    try:
        pp = PostProcessing( h5_files[-1], params["data_path"], device = "gpu")
    except:
        print("You haven't trained anything?")
        continue

    pfile = open(os.path.join(params["save_folder"],  "results.txt"), "w")
    pfile.write("Overall perfomance: \n")
    accuracy_test, trainable_count = pp.evaluate_overall(device = "gpu")
    pfile.write("Accuracy: {} \nTrainable parameters: {} \n\n".format(round(accuracy_test,2)*100, trainable_count) )
    
    pfile.write("Performance per class:\n")
    beam_accuracy, tissue_accuracy, bone_accuracy = pp.evaluate_perclass()
    pfile.write(" Open beam: {} \n Soft Tissue: {} \n Bone: {}\n\n".format(round(beam_accuracy,2)*100, round(tissue_accuracy,2)*100, round(bone_accuracy,2)*100))
    
    pfile.write("True Positives and False Positives:\n")
    tp, fp = pp.tpfp()
    pfile.write(" TP: {} \n FP {} \n\n ".format(round(tp,2)*100, round(fp,2)*100))
    
    pfile.write("Threshold 90% \n")
    thresh90 = pp.thresholding(0.9)
    tp90, fp90 = pp.tpfp(thresh90)
    pfile.write(" TP90: {} \n FP90 {}\n \n ".format(round(tp90,2)*100, round(fp90,2)*100))
    
    pfile.write("Threshold 99% \n")
    thresh99 = pp.thresholding(0.99)
    tp99, fp99 = pp.tpfp(thresh99)
    pfile.write(" TP99: {} \n FP99 {}\n \n ".format(round(tp99,2)*100, round(fp99,2)*100))
    
    pfile.close()
   
    lc_fig, lc_ax = pp.learning_curve(os.path.join(params["save_folder"], params["name"] + ".csv"))
    lc_fig.savefig(os.path.join(params["save_folder"], "learning_curve.png"))
    
    rc_fig, rc_ax = pp.ROC_curve()
    rc_fig.savefig( os.path.join(params["save_folder"] , "roc_curve.png") )
    
    


Warning, folder exists! Delete? (y/n) y


TypeError: ignored

In [29]:
for file in param_files:
    params = json.load(open(file,'r'))
    #params["save_folder"]

    training = TrainingClass.TrainingClass(**params)

In [49]:
try:
  training.fit()
except:
  print("\n Dying... \n")


 Dying... 



In [32]:
import numpy as np
import TrainingClass
import json
import os
import sys
from PostProcessing import PostProcessing
import glob
import shutil
import webbrowser
import time
#from Killer import GracefulKiller
#killer = GracefulKiller()

param_files = glob.glob("aug*.txt")
print("I will train on all these parameter files:\n")
print(*param_files, sep = "\n")

#print("Opening tensorboard... \n")
#tb_url = "http://127.0.0.1:7007/"
#webbrowser.open(tb_url)

for file in param_files:
    params = json.load(open(file,'r'))

    save_folder = params["save_folder"]
    if(os.path.isdir(save_folder)):
        rm_folder = input("Warning, folder exists! Delete? (y/n) ")
        if(rm_folder == "y"):
            shutil.rmtree(save_folder)
        else:
            sys.exit()
            
    os.mkdir(save_folder)

    #tensorboard stuff
    #tbdir = os.path.join(save_folder, "tboard")
    #os.mkdir(tbdir)
    #os.system("killall tensorboard")
    #os.system("tensorboard --logdir=" + tbdir + " --port=7007 &")
    training = TrainingClass.TrainingClass(**params)
    try:
        training.fit()
    except:
        print("\n Dying... \n")
        
    print("Running post training analysis...\n")
        
    h5_files = np.sort(glob.glob(os.path.join(params["save_folder"], "*.h5")))
    try:
        pp = PostProcessing( h5_files[-1], params["data_path"], device = "gpu")
    except:
        print("You haven't trained anything?")
        continue

    pfile = open(os.path.join(params["save_folder"],  "results.txt"), "w")
    pfile.write("Overall perfomance: \n")
    accuracy_test, trainable_count = pp.evaluate_overall(device = "gpu")
    pfile.write("Accuracy: {} \nTrainable parameters: {} \n\n".format(round(accuracy_test,2)*100, trainable_count) )
    
    pfile.write("Performance per class:\n")
    beam_accuracy, tissue_accuracy, bone_accuracy = pp.evaluate_perclass()
    pfile.write(" Open beam: {} \n Soft Tissue: {} \n Bone: {}\n\n".format(round(beam_accuracy,2)*100, round(tissue_accuracy,2)*100, round(bone_accuracy,2)*100))
    
    pfile.write("True Positives and False Positives:\n")
    tp, fp = pp.tpfp()
    pfile.write(" TP: {} \n FP {} \n\n ".format(round(tp,2)*100, round(fp,2)*100))
    
    pfile.write("Threshold 90% \n")
    thresh90 = pp.thresholding(0.9)
    tp90, fp90 = pp.tpfp(thresh90)
    pfile.write(" TP90: {} \n FP90 {}\n \n ".format(round(tp90,2)*100, round(fp90,2)*100))
    
    pfile.write("Threshold 99% \n")
    thresh99 = pp.thresholding(0.99)
    tp99, fp99 = pp.tpfp(thresh99)
    pfile.write(" TP99: {} \n FP99 {}\n \n ".format(round(tp99,2)*100, round(fp99,2)*100))
    
    pfile.close()
   
    lc_fig, lc_ax = pp.learning_curve(os.path.join(params["save_folder"], params["name"] + ".csv"))
    lc_fig.savefig(os.path.join(params["save_folder"], "learning_curve.png"))
    
    rc_fig, rc_ax = pp.ROC_curve()
    rc_fig.savefig( os.path.join(params["save_folder"] , "roc_curve.png") )
    
    


I will train on all these parameter files:




In [57]:
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1Wel_XsyE7HcEq0TkZWI61GABO4jOtj9C',
                                    dest_path='./dataset.hdf5')
gdd.download_file_from_google_drive(file_id='1cePD5E-T9mr5W0xPGuzEnUt8Glpvn23U',
                                    dest_path='./model.h5')

In [58]:
hdf5_path = "./dataset.hdf5" ## this is our h5 file containing training and testing data
dataset = h5py.File(hdf5_path , 'r')

classes = 3
test_images = dataset['test_img'][:]
print(test_images.shape)
no_images, height, width, channels = test_images.shape


print(dataset['test_label'][:].shape)
test_labels =dataset['test_label'][:].reshape(-1,height*width, classes )
print(test_labels.shape)
dataset.close()

(22, 200, 200, 1)
(22, 40000, 3)
(22, 40000, 3)


In [35]:
training.model.predict(test_images)

NameError: ignored